exerciseにあたってのメモ

インプットはtopicのみでsystem_templateで以下のようにするつもりだった。
- system_template = "you are a quiz producer who specializes in {topics}"

上のsystem_templateはコメントにして以下のものとした。
- system_template = "you are a quiz producer who specializes in a historical event"

ペアとなるhuman_templateは以下。
- human_template = "i want you make only one quiz which ask a specific date of a historical event in the following format.\n{format_instruction}"


質問への回答はdatetimeフォーマットで、datatimeoutparseを使う

ユーザーがタイプする年、月、日をdatetimeフォーマットに変換

2つのdatetimeフォーマットのデータを比較して差を提示


In [2]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash",convert_system_message_to_human=True,temperature=0)
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers import DatetimeOutputParser
from pydantic import BaseModel, Field
from datetime import datetime

In [3]:
class Quiz(BaseModel):
  question: str = Field(description="Question about a specific date of an event")
  answer: datetime | None

In [4]:
parser = PydanticOutputParser(pydantic_object=Quiz)

In [5]:
# system_template = "you are a quiz producer who specializes in {topics}"
system_template = "you are a quiz producer who specializes in a historical event"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "i want you make only one quiz which ask a specific date of a historical event in the following format.\n{format_instruction}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

request = chat_prompt.format_prompt(topics="world war 2", format_instruction=parser.get_format_instructions()).to_messages()
    


In [ ]:
chat1 = ChatGoogleGenerativeAI(model="gemini-1.5-flash",convert_system_message_to_human=True)
result1 = chat1.invoke(request)

In [7]:
print(result1.content)

```json
{
  "question": "On what date did the signing of the Declaration of Independence take place?",
  "answer": "1776-07-04"
}
```


In [8]:
type(parser.parse(result1.content))

__main__.Quiz

In [9]:
print(parser.parse(result1.content).question)

On what date did the signing of the Declaration of Independence take place?


In [10]:
output_parser = DatetimeOutputParser()

In [47]:
# template_text = "on what date did {request} end?\n{format_instructions}"
# human_prompt=HumanMessagePromptTemplate.from_template(template_text)

In [48]:
# chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [49]:
# topics = "world war 2"
# topics = "japanese meiji period"
# request = chat_prompt.format_prompt(request=topics,
#                    format_instructions=output_parser.get_format_instructions()
#                    ).to_messages();

In [ ]:
# result = chat.invoke(request);


In [51]:
# result.content

In [52]:
# ai_date = output_parser.parse(str(result.content))

In [55]:
print("on what date did world war 2 end?")
y = input('enter the year:')
m = input('enter the month:')
d = input('enter the date:')
input_user_date = y+'/'+m+'/'+d
user_date = datetime.strptime(input_user_date, '%Y/%m/%d')

print(user_date, " and ", type(user_date))


on what date did world war 2 end?
1980-08-08 00:00:00  and  <class 'datetime.datetime'>


In [56]:
# print('the difference between the dates: ', ai_date-user_date)

the difference between the dates:  -24846 days, 0:00:00


In [57]:
# from langchain.prompts import (
#     ChatPromptTemplate,
#     PromptTemplate,
#     SystemMessagePromptTemplate,
#     AIMessagePromptTemplate,
#     HumanMessagePromptTemplate,
# )
# from datetime import datetime
# # from langchain_openai import OpenAI
# from langchain.output_parsers import DatetimeOutputParser
# # from langchain.chat_models import ChatOpenAI

In [58]:
# from langchain_google_genai import ChatGoogleGenerativeAI
# chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0)

In [44]:
class HistoryQuiz():
    
    def create_history_question(self,topic):
        '''
        This method should output a historical question about the topic that has a date as the correct answer.
        For example:
        
            "On what date did World War 2 end?"
            
        '''
        from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
        from langchain_google_genai import ChatGoogleGenerativeAI
        chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash",convert_system_message_to_human=True,temperature=0)
        from langchain.output_parsers import PydanticOutputParser
        from langchain.output_parsers import DatetimeOutputParser
        from pydantic import BaseModel, Field
        from datetime import datetime
       
        class Quiz(BaseModel):
            question: str = Field(description="Question about a specific date of an event")
            answer: datetime | None
        
        parser = PydanticOutputParser(pydantic_object=Quiz)
        
        # system_template = "you are a quiz producer who specializes in {topics}"
        system_template = "you are a quiz producer who specializes in a historical event"
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        human_template = "i want you make only one quiz which ask a specific date of a historical event in the following format.\n{format_instruction}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

        request = chat_prompt.format_prompt(format_instruction=parser.get_format_instructions()).to_messages()
            
    
        chat1 = ChatGoogleGenerativeAI(model="gemini-1.5-flash",convert_system_message_to_human=True)
        result1 = chat1.invoke(request) 
        print("create_history_question()", result1)
        question = parser.parse(result1.content).question
        
        return question
    
    def get_AI_answer(self,question):
        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!
        
        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''
         
        class Answer(BaseModel):
            answer: datetime | None
        parser = PydanticOutputParser(pydantic_object=Answer)
        
        system_template = "you are a quiz professional who specialized in a historical event"
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        human_template = "i want you answer my question, {question}, whick ask a specific date of a historical event in the following format.\n{format_instruction}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

        request = chat_prompt.format_prompt(question=question,format_instruction=parser.get_format_instructions()).to_messages()
            
    
        chat1 = ChatGoogleGenerativeAI(model="gemini-1.5-flash",convert_system_message_to_human=True)
        result1 = chat1.invoke(request) 
        correct_datetime = parser.parse(result1.content).answer

        return correct_datetime
    
    def get_user_answer(self,question):
        '''
        This method should grab a user answer and convert it to datetime. It should collect a Year, Month, and Day.
        You can just use input() for this.
        '''
        print(question)
        y = input('enter the year:')
        m = input('enter the month:')
        d = input('enter the date:')
        input_user_date = y+'/'+m+'/'+d
        user_datetime = datetime.strptime(input_user_date, '%Y/%m/%d')        

        
        return user_datetime
        
        
    def check_user_answer(self,user_answer,ai_answer):
        '''
        Should check the user answer against the AI answer and return the difference between them
        '''
        # print or return the difference between the answers here!
        print("The difference between the dates is: ", user_answer-ai_answer)
        

### Example Usage

Feel free to expand or edit this project. To keep things simple we have every method return an object that will then feed into a new method!

In [45]:
quiz_bot = HistoryQuiz()

In [ ]:
# topic is not used in the function
question = quiz_bot.create_history_question(topic='World War 2')


In [47]:
question

'On what date did the Berlin Wall fall?'

In [ ]:
ai_answer = quiz_bot.get_AI_answer(question)

In [49]:
# Day After Pearl Harbor
ai_answer

datetime.datetime(1989, 11, 9, 0, 0)

In [50]:
user_answer = quiz_bot.get_user_answer(question)

On what date did the Berlin Wall fall?


In [ ]:
user_answer

datetime.datetime(1941, 12, 1, 0, 0)

In [ ]:
quiz_bot.check_user_answer(user_answer,ai_answer)

The difference between the dates is: -7 days, 0:00:00
